In [12]:
import os
import pandas as pd
import sys
from pathlib import Path

path = os.getcwd()
p = Path(path).parents[1]
# print("Current Directory", path)
# prints parent directory
directory_path = os.path.abspath(p) + "\\data"
directory_path
# os.path.abspath(os.path.join(path, os.pardir))
# directory_path = os.path.abspath(os.path.join(path, os.pardir)) + "\\data"
file_path = directory_path.replace("\\", "/")
all_csv_list = os.listdir(directory_path)

In [13]:
for filename in all_csv_list:
    if "2020" in filename:
        r_df = pd.read_csv(os.path.join(file_path, filename), compression="zip")

In [157]:
r_df_10 = r_df.loc[(r_df["Departure"] >= "2020-10-01") & (r_df["Departure"] <= "2020-11-01")]
r_df_10_group = r_df_10.groupby(['Departure station']).size().reset_index(name='ride_count')

In [161]:
r_df_group = r_df.groupby(['Departure station']).size().reset_index(name='ride_count').sort_values(["ride_count"], ascending=False)
r_df_group

,Departure station,ride_count
25,0028 Davie & Beach,10886
54,0066 Anderson & 2nd,10062
153,0209 Stanley Park - Information Booth,9988
81,0102 Stanley Park - Second Beach North,8981
12,0011 Ontario & Seawall,8712
...,...,...
222,0991 HQ Workshop,9
220,0985 Yard - Long Term Storage,8
221,0986 Quebec Yard - Serviced,5
223,0995 Workshop - On Deck,2


In [170]:
import numpy as np

norm = np.linalg.norm(r_df_group["ride_count"])
r_df_group["ride_count"] = round(((r_df_group["ride_count"]/norm) * 1000), 4)
r_df_group

,Departure station,ride_count
25,0028 Davie & Beach,214.1566
54,0066 Anderson & 2nd,197.9463
153,0209 Stanley Park - Information Booth,196.4906
81,0102 Stanley Park - Second Beach North,176.6802
12,0011 Ontario & Seawall,171.3882
...,...,...
222,0991 HQ Workshop,0.1771
220,0985 Yard - Long Term Storage,0.1574
221,0986 Quebec Yard - Serviced,0.0984
223,0995 Workshop - On Deck,0.0393


In [89]:
import urllib.request, json 
with urllib.request.urlopen("https://vancouver-gbfs.smoove.pro/gbfs/en/station_information.json") as url:
    s_data = json.loads(url.read().decode())
s_df = pd.json_normalize(s_data['data']['stations'])
s_df['station'] = s_df['station_id'] + " " + s_df['name'] 
s_df

,station_id,name,lat,lon,capacity,station
0,0001,10th & Cambie,49.262487,-123.114397,35,0001 10th & Cambie
1,0002,Burrard Station (Melville & Dunsmuir),49.285871,-123.121050,28,0002 Burrard Station (Melville & Dunsmuir)
2,0004,Yaletown-Roundhouse Station,49.274566,-123.121817,16,0004 Yaletown-Roundhouse Station
3,0005,Dunsmuir & Beatty,49.279764,-123.110154,26,0005 Dunsmuir & Beatty
4,0006,Olympic Village Station,49.266314,-123.116011,26,0006 Olympic Village Station
...,...,...,...,...,...,...
204,0297,6th & Prince Edward,49.265242,-123.095577,16,0297 6th & Prince Edward
205,0298,6th & Carolina,49.265213,-123.090785,16,0298 6th & Carolina
206,0300,Bute & Barclay,49.284893,-123.128685,14,0300 Bute & Barclay
207,0305,Georgia & Homer,49.280787,-123.115271,18,0305 Georgia & Homer


In [131]:
import json
geo_file = file_path + "/tmp/geocodings.pickle"
gps_data = pd.read_pickle(geo_file)
stationid, station, zip, lon, lat = [], [], [], [], []
# print(type(gps_data))
for row in gps_data.items():
    station.append(row[0])
    stationid.append(row[0].split(" ")[0])
    zip.append(row[1][0])
    lat.append(float(str(row[1][1])[0:10]))
    lon.append(float(str(row[1][2])[0:12]))
    
    # print("station id: {},    station name: {}".format(station, station))
print("length station {}, zip: {}, lon: {}, lat:{}".format(len(station), len(zip), len(lon), len(lat)))


length station 254, zip: 254, lon: 254, lat:254


In [132]:
station_df = pd.DataFrame()
station_df['station_id'] = stationid
station_df['station'] = station
station_df['zip'] = zip
station_df["lon"] = lon
station_df["lat"] = lat
station_df = station_df.sort_values(["station_id"], ascending=True)

In [133]:
station_df.reset_index()

,index,station_id,station,zip,lon,lat
0,26,0001,0001 10th & Cambie,V5Y1R3,-123.114397,49.262487
1,1,0002,0002 Burrard Station,V6C0A4,-123.121050,49.285871
2,230,0002,0002 Burrard Station (Melville & Dunsmuir),V6C,-123.119399,49.285552
3,46,0004,0004 Yaletown-Roundhouse Station,V6B0H8,-123.121817,49.274566
4,80,0005,0005 Dunsmuir & Beatty,V6B,-123.110154,49.279764
...,...,...,...,...,...,...
249,128,137-0188,137-0188 Granville Island Public Market,V6H3R9,-123.135215,49.272701
250,20,35-0601,35-0601 Olympic Village Station,V5Z,-123.115620,49.266540
251,131,41-0075,41-0075 10th & Arbutus,V6J3Y2,-123.152425,49.262866
252,126,73-0151,73-0151 Pine & 4th,L9Y2P2,-80.217808,44.496745


In [134]:
station_df.to_csv(r'np.txt', header=None, index=None, sep='\t', mode='a')

In [100]:


merged_df2 = pd.merge(left=r_df_group, right=s_df, how='left', left_on='Departure station', right_on='station')

merged_df2

,Departure station,ride_count,station_id,name,lat,lon,capacity,station
0,0001 10th & Cambie,3983,0001,10th & Cambie,49.262487,-123.114397,35.0,0001 10th & Cambie
1,0002 Burrard Station,824,NaN,NaN,NaN,NaN,NaN,NaN
2,0002 Burrard Station (Melville & Dunsmuir),2104,0002,Burrard Station (Melville & Dunsmuir),49.285871,-123.121050,28.0,0002 Burrard Station (Melville & Dunsmuir)
3,0004 Yaletown-Roundhouse Station,3958,0004,Yaletown-Roundhouse Station,49.274566,-123.121817,16.0,0004 Yaletown-Roundhouse Station
4,0005 Dunsmuir & Beatty,6312,0005,Dunsmuir & Beatty,49.279764,-123.110154,26.0,0005 Dunsmuir & Beatty
...,...,...,...,...,...,...,...,...
220,0985 Yard - Long Term Storage,8,NaN,NaN,NaN,NaN,NaN,NaN
221,0986 Quebec Yard - Serviced,5,NaN,NaN,NaN,NaN,NaN,NaN
222,0991 HQ Workshop,9,NaN,NaN,NaN,NaN,NaN,NaN
223,0995 Workshop - On Deck,2,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
rr2 = r_df_group.merge(right=station_df, left_on='Departure station', right_on='station')
rr2

,Departure station,ride_count,station_id,station,zip,lon,lat
0,0001 10th & Cambie,3983,0001,0001 10th & Cambie,V5Y1R3,-123.114397,49.262487
1,0002 Burrard Station,824,0002,0002 Burrard Station,V6C0A4,-123.121050,49.285871
2,0002 Burrard Station (Melville & Dunsmuir),2104,0002,0002 Burrard Station (Melville & Dunsmuir),V6C,-123.119399,49.285552
3,0004 Yaletown-Roundhouse Station,3958,0004,0004 Yaletown-Roundhouse Station,V6B0H8,-123.121817,49.274566
4,0005 Dunsmuir & Beatty,6312,0005,0005 Dunsmuir & Beatty,V6B,-123.110154,49.279764
...,...,...,...,...,...,...,...
216,0981 Workshop - Service Complete,389,0981,0981 Workshop - Service Complete,L7L5H7,-79.763537,43.390667
217,0985 Quebec Yard - To Service,41,0985,0985 Quebec Yard - To Service,H2Z1X4,-73.563392,45.506058
218,0985 Yard - Long Term Storage,8,0985,0985 Yard - Long Term Storage,V2T,-122.383601,49.021526
219,0986 Quebec Yard - Serviced,5,0986,0986 Quebec Yard - Serviced,J0K1B0,-73.483220,45.971435


In [171]:
merged_df = pd.merge(left=r_df_group, right=station_df, how='left', left_on='Departure station', right_on='station')

merged_df = merged_df[merged_df["lon"] < -121]

,Departure station,ride_count,station_id,station,zip,lon,lat
0,0028 Davie & Beach,214.1566,0028,0028 Davie & Beach,V6G,-123.142139,49.287528
1,0066 Anderson & 2nd,197.9463,0066,0066 Anderson & 2nd,V6H,-123.137820,49.269361
2,0209 Stanley Park - Information Booth,196.4906,0209,0209 Stanley Park - Information Booth,V6G3E2,-123.131478,49.297769
3,0102 Stanley Park - Second Beach North,176.6802,0102,0102 Stanley Park - Second Beach North,V6G3E2,-123.150433,49.295340
4,0011 Ontario & Seawall,171.3882,0011,0011 Ontario & Seawall,V5Y,-123.104207,49.271721
...,...,...,...,...,...,...,...
220,0991 HQ Workshop,0.1771,NaN,NaN,NaN,NaN,NaN
221,0985 Yard - Long Term Storage,0.1574,0985,0985 Yard - Long Term Storage,V2T,-122.383601,49.021526
222,0986 Quebec Yard - Serviced,0.0984,0986,0986 Quebec Yard - Serviced,J0K1B0,-73.483220,45.971435
223,0995 Workshop - On Deck,0.0393,0995,0995 Workshop - On Deck,N6P,-81.291349,42.872283


In [175]:
merged_df.describe()


,ride_count,lon,lat
count,225.000000,221.000000,221.000000
mean,50.026797,-121.083394,49.116698
std,44.163705,9.303430,0.930525
min,0.039300,-123.157443,42.872283
25%,18.807100,-123.129935,49.263242
50%,36.001000,-123.114494,49.273355
75%,64.152600,-123.089413,49.280876
max,214.156600,-64.237719,52.115275


In [172]:
# merged_df = merged_df.dropna()
merged_df[["ride_count"]].describe()

merged_df_sorted = merged_df.sort_values(["ride_count"], ascending=False).reset_index()

In [173]:
merged_df_sorted

,index,Departure station,ride_count,station_id,station,zip,lon,lat
0,0,0028 Davie & Beach,214.1566,0028,0028 Davie & Beach,V6G,-123.142139,49.287528
1,1,0066 Anderson & 2nd,197.9463,0066,0066 Anderson & 2nd,V6H,-123.137820,49.269361
2,2,0209 Stanley Park - Information Booth,196.4906,0209,0209 Stanley Park - Information Booth,V6G3E2,-123.131478,49.297769
3,3,0102 Stanley Park - Second Beach North,176.6802,0102,0102 Stanley Park - Second Beach North,V6G3E2,-123.150433,49.295340
4,4,0011 Ontario & Seawall,171.3882,0011,0011 Ontario & Seawall,V5Y,-123.104207,49.271721
...,...,...,...,...,...,...,...,...
220,220,0991 HQ Workshop,0.1771,NaN,NaN,NaN,NaN,NaN
221,221,0985 Yard - Long Term Storage,0.1574,0985,0985 Yard - Long Term Storage,V2T,-122.383601,49.021526
222,222,0986 Quebec Yard - Serviced,0.0984,0986,0986 Quebec Yard - Serviced,J0K1B0,-73.483220,45.971435
223,223,0995 Workshop - On Deck,0.0393,0995,0995 Workshop - On Deck,N6P,-81.291349,42.872283


In [168]:
merged_df_sorted = merged_df_sorted.dropna()

In [174]:
station_list = []

# Iterating through the json
# list
for idx, row in merged_df_sorted.iterrows():
    v1, v2 = str(row['lat']), str(row['lon'])
    v3 = row['ride_count']
    st = "location: new google.maps.LatLng({}, {}), weight: {}".format(v1, v2, v3)
    station_list.append("{" +st + "},")

with open('station_google_cmd.txt', 'w') as f:
    for station in station_list:
        f.write(station)
        f.write('\n')

,station_id,name,lat,lon,capacity
0,0001,10th & Cambie,49.262487,-123.114397,35
1,0002,Burrard Station (Melville & Dunsmuir),49.285871,-123.121050,28
2,0004,Yaletown-Roundhouse Station,49.274566,-123.121817,16
3,0005,Dunsmuir & Beatty,49.279764,-123.110154,26
4,0006,Olympic Village Station,49.266314,-123.116011,26
...,...,...,...,...,...
204,0297,6th & Prince Edward,49.265242,-123.095577,16
205,0298,6th & Carolina,49.265213,-123.090785,16
206,0300,Bute & Barclay,49.284893,-123.128685,14
207,0305,Georgia & Homer,49.280787,-123.115271,18
